# DSGA 1018 - Lab 2

Please submit the code and a pdf of the output.

Your work will be evaluated based on the code and plots. You don't need to write down your answers to these questions in the text blocks. Everything that will be graded is indicated by the "TODO".

Due: 2021-10-05 23:59pm (Next Tuesday midnight)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from statsmodels.tsa.stattools import acf, ccf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics import utils
import statsmodels.api as sm
import pandas as pd
from statsmodels.graphics.api import qqplot

## Apply AR Model

In this example, we will fit an AR(p) model to the SunActivity data, which denotes the number of sunspots for each year.

We will determine p, fit the model, compute the roots and the lag 0 to p components of the ACF.

Wikipedia for sunspots: https://en.wikipedia.org/wiki/Sunspot

The code in this section is selected from the tutorial specified in the reference section.

In [ ]:
dta = sm.datasets.sunspots.load_pandas().data
dta.index = pd.Index(sm.tsa.datetools.dates_from_range('1700', '2008'))
del dta["YEAR"]
dta.plot(figsize=(12,8))
plt.show()

#### ACF & PACF

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(dta.values.squeeze(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(dta, lags=40, ax=ax2)
plt.show()

#### Fit AR Model of order p

In [ ]:
# TODO: chose p appropriately
p = 

arma_mod = sm.tsa.ARMA(dta, (p,0)).fit(disp=False)
print(arma_mod.params)

# TODO: predict ACF of model at lag 0, 1, ..., p
rho = np.zeros(p+1)
    
# TODO: compute roots
roots = np.zeros(2)

print('roots: ', roots)

#### prediction

In [ ]:
# TODO: try to predict further into the future by increasing tsteps
tsteps=

fig, ax = plt.subplots(figsize=(12, 8))
ax = dta.ix['1900':].plot(ax=ax)
T = np.arange(1930, 2010, tsteps)
for tt in range(len(T)-1):
    fig = arma_mod.plot_predict(np.str(T[tt]), np.str(T[tt+1]), dynamic=True, ax=ax, plot_insample=False)
plt.show()

#### plot ACF and PACF

In [ ]:
predict_sunspots = arma_mod.predict('1950', '2012', dynamic=True)

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(dta.values.squeeze(), lags=40, ax=ax1)
ax1.plot(np.arange(p+1), rho, 'xr', label='prediction')
ax1.legend()
plt.show()

## Implement AR model from Scratch

### Section I: Implementing the AR Model

Recall that the negative log-likelihood function takes as input the parameter values and returns the negative log probability
of the observed data, under the assumption that those were the parameters used to generate the data.

For an AR(p) model, we have:

$$ NLL(\phi_1, \phi_2, \ldots, \phi_p, \sigma ~; x_1, x_2, \ldots, x_n) = \sum_{t=p+1}^n \left( \log \left( \sigma \sqrt{2 \pi} \right) + \frac{1}{2} \cdot \left( \frac{x_t - \left( \sum\limits_{i=1}^p \phi_i x_{t-i} \right)}{\sigma} \right)^2 \right) $$

Optional: We use Nelder-Mead Algorithm to find the minimum of the loss. Check this link if you're interested (https://machinelearningmastery.com/how-to-use-nelder-mead-optimization-in-python/)

In [ ]:
class ARModel:
    """Class that implements an ARMA Model. Its functions are as follows:
    1. Maximum Likelihood estimation of parameters
    2. Inference/prediction of future states
    3. Data simulation
    """
    def __init__(self, p, data, p_params = None, sigma = None):
        """Initialize the network state
        @param p: the number of time steps to include in the AR process
        @param p_params: the initialization for the AR parameters
        """
        if (p_params is None):
            p_params = np.zeros(p)
        if (sigma is None):
            sigma = 1
            
        assert p == len(p_params)
        
        #assign parameter values
        self.p = p
        self.p_params = p_params
        self.sigma = sigma
        #store the data within the object
        self.data = data
    
    def loss(self, params):
        """
        params: array of parameters, elements 0:p = p_params, element p = sigma
        returns: loss
        """
        assert len(params) == self.p + 1
        N = self.data.shape[0]
        p_params = params[0:self.p]
        sigma = params[self.p]
        loss = 0
        
        #TODO: calculate the NLL of the data for the purposes of optimization and store it in loss
        
        return loss

    def fit(self):
        # Minimize the loss function, given the dataset
        params = np.concatenate((self.p_params, np.array([self.sigma])))
        res = minimize(self.loss, params, method='nelder-mead',
               options={'xatol': 1e-8, 'disp': True})
        self.p_params = res.x[0:self.p]
        self.sigma = res.x[self.p]
        
    def predict(self,data, N):
        """Method that predicts N timesteps in the future given input data
        @params data: p data points used to form the prediction
        @params N: number of time steps to predict in the future
        
        returns:
        prediction: predicted future value
        conf: variance of the estimated future value
        """
        assert len(data) == self.p
        predction = np.zeros(N)
        conf = np.zeros(N)
        
        #TODO: predict N time steps in advance, given an input. 
        #The inference can be specific to your choice of p, no need to worry about general inference here

        return prediction, conf
        
    def simulate(self,N):
        """Method that stimulates data given the p_params and q_params
        @param N: number of datapoints to simulate
        returns: N sampled datapoints
        """
        transient = 100 # length of time to run the simulation to wash out initial conditions
        w_t = self.sigma * np.random.normal(size = (N + transient,))
        x_t = np.zeros(N + transient)
        
        # TODO: generate data x_t given the parameters and white noise w_t
        
        return x_t[transient::] #discard the transient when returning simulated data    

### Section II: Fitting the AR Model

In this section, we will load some data from an unknown source, look at its ACF and PACF
plots to determine an appropriate AR(p) order, and fit the AR(p) model to the data to determine
the coefficients of the AR model as well as the standard deviation of the driving white noise process.

In [ ]:
data = np.load("lab_2_data.npy")

#plot the acf of the data
lag = 20
plt.plot(data)
plot_acf(x=data, lags=lag, title="ACF w_t")
plt.show()

#plot the pacf of the data
plt.figure()
sm.graphics.tsa.plot_pacf(data, lags=40)
plt.show()

In [ ]:
# fit the model
p = 0 #TODO: choose a 'p' value
data_fitter = ARModel(p, data, p_params = np.array([]), sigma = 0.5) # set p_params and sigma to an educated guess for parameter values
data_fitter.fit()
print('lambda = ' + str(data_fitter.p_params))
print('sigma = ' + str(data_fitter.sigma))

### Section III: Simulating data

Now, we will use our fitted model to simulate a run of the AR model.

In [ ]:
#Generate 1000 samples from the fit model
data_2 = data_fitter.simulate(1000)

#Compare the ACF from the fit model to the data ACF
lag = 20

plot_acf(x=data, lags=lag, title="ACF data")
plot_acf(x=data_2, lags=lag, title="ACF data_2")
plt.show()

#Compare the PACF from the fit model to the data ACF
plt.figure()
sm.graphics.tsa.plot_pacf(data, lags=20)
plt.title('PACF Data')
sm.graphics.tsa.plot_pacf(data_2, lags=20)
plt.title('PACF Data 2')
plt.show()

### Section IV: Using the AR Model for prediction

Finally, we will use some of the provided data as a starting point and predict the next 20 values
based on our AR model's fitted parameters. This will be repeated for each of various starting points.

In [ ]:
#for each of the given data points, generate predictions 20 time steps into the future

data_prediction = data[0:100:25]
predictions = np.zeros((len(data_prediction), 20))
mse = np.zeros((len(data_prediction), 20))
for ii in range(0, len(data_prediction)):
    for jj in range(0,20):
        #for each data point, predict for each of 20 time steps
        predictions[ii,jj], mse[ii,jj] = data_fitter.predict(data_prediction[[ii]],jj)

In [ ]:
#plot the MSE bars of the estimate
plt.figure()
for ii in range(0, len(data_prediction)):
    plt.errorbar(np.arange(0,20), predictions[ii,:], yerr = mse[ii,:])
plt.legend(data_prediction)
plt.show()